In [2]:
# Instalar o Terraform
!apt-get update && apt-get install -y wget unzip
!wget https://releases.hashicorp.com/terraform/1.10.5/terraform_1.10.5_linux_amd64.zip
!unzip terraform_1.10.5_linux_amd64.zip
!mv terraform /usr/local/bin/

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,321 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,708 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

In [3]:
# Verificar a instalação
!terraform -v

Terraform v1.10.5
on linux_amd64

Your version of Terraform is out of date! The latest version
is v1.11.0. You can update by downloading from https://www.terraform.io/downloads.html


In [4]:
# Importar credenciais do GCP
import json
from google.colab import files
uploaded = files.upload()

Saving petadota-2c8ab5b413fa.json to petadota-2c8ab5b413fa.json


In [7]:
%%writefile infra.tf

provider "google" {
  credentials = file("${path.module}/petadota-2c8ab5b413fa.json")
  project     = "167645809807"
  region      = "us-central1"
  zone        = "us-central1-a"
}

resource "google_compute_instance" "petadota_vm" {
  name         = "petadota-ativ5-instance"
  machine_type = "e2-micro"

  boot_disk {
    initialize_params {
      image = "debian-cloud/debian-11"
    }
  }

  network_interface {
    network = "default"

    access_config {
    }
  }
}


Overwriting infra.tf


In [8]:
# Inicializar o Terraform.
!terraform init

# Planejar a criação de recursos
!terraform plan

# Aplicar as configurações e provisionar o bucket
!terraform apply -auto-approve

Initializing the backend...
Initializing provider plugins...
- Reusing previous version of hashicorp/google from the dependency lock file
- Using previously-installed hashicorp/google v5.45.2

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.
google_storage_bucket.petadota_bucket: Refreshing state... [id=bucket-petadota-completo]

Terraform used the selected providers to generate the following execution plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # google_compute_instance.petadota_vm will be created
  + resource "google_comput

In [9]:
# Instalar o Ansible
!apt-get update && apt-get install -y software-properties-common
!apt-add-repository --yes --update ppa:ansible/ansible
!apt-get install -y ansible

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 6,555 B in 1s (5,179 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... D

In [10]:
# Checar a instalação
!ansible --version

ansible [core 2.17.9]
  config file = /etc/ansible/ansible.cfg
  configured module search path = ['/root/.ansible/plugins/modules', '/usr/share/ansible/plugins/modules']
  ansible python module location = /usr/lib/python3/dist-packages/ansible
  ansible collection location = /root/.ansible/collections:/usr/share/ansible/collections
  executable location = /usr/bin/ansible
  python version = 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0] (/usr/bin/python3)
  jinja version = 3.1.5
  libyaml = True


In [11]:
# Importar chave privada SSH
import json
from google.colab import files
uploaded = files.upload()

Saving key.pem to key.pem


In [12]:
# Corrigir permissões da chave privada se necessário
!chmod 600 key.pem

In [13]:
%%writefile hosts

[dbservers]
34.122.19.27 ansible_ssh_user=danib ansible_ssh_private_key_file=key.pem ansible_ssh_common_args='-o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null'

Writing hosts


In [18]:
%%writefile db-playbook.yml
- hosts: all
  become: yes
  tasks:
    - name: Instalar debconf-utils
      apt:
        name: debconf-utils
        state: present

    - name: Atualizar o repositório apt
      apt:
        update_cache: yes

    - name: Instalar mysqlclient para Python
      apt:
        name: python3-mysqldb
        state: present

    - name: Instalar o MariaDB server
      apt:
        name: mariadb-server
        state: present

    - name: Iniciar e habilitar o serviço MariaDB
      service:
        name: mariadb
        state: started
        enabled: yes

    - name: Configurar senha de root do MariaDB
      debconf:
        name: 'mariadb-server'
        question: 'mariadb-server/root_password'
        value: 'petAdot@2025'
        vtype: 'password'

    - name: Configurar senha de root do MariaDB (confirmation)
      debconf:
        name: 'mariadb-server'
        question: 'mariadb-server/root_password_again'
        value: 'petAdot@2025'
        vtype: 'password'

    - name: Certificar que o MariaDB está rodando
      service:
        name: mariadb
        state: started
        enabled: yes

    - name: Criar banco de dados petadota_db
      mysql_db:
        name: petadota_db
        state: present

    - name: Criar usuário petadmin
      mysql_user:
        name: petadmin
        password: "petAdot@2025#Admin"
        priv: "petadota_db.*:ALL"
        state: present

Overwriting db-playbook.yml


In [19]:
!ansible-playbook -i hosts db-playbook.yml --key-file key.pem -vv

/usr/lib/python3/dist-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
/usr/lib/python3/dist-packages/paramiko/transport.py:261: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,
ansible-playbook [core 2.17.9]
  config file = /etc/ansible/ansible.cfg
  configured module search path = ['/root/.ansible/plugins/modules', '/usr/share/ansible/plugins/modules']
  ansible python module location = /usr/lib/python3/dist-p